## 常明杰的毕业设计作品

<h2>基于MobileNetV3的果树病虫害识别</h2>

首先，我们导入需要的包

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)
import os
import shutil
import matplotlib.pyplot as plt

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h2>加载数据和预处理</h2>
<h4>下面，加载数据，看一下我们需要处理什么</h4>

In [ ]:
train = pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')
test = pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')
print(train)
target = train[['healthy', 'multiple_diseases', 'rust', 'scab']]
print(target)

test_ids = test['image_id']
print(test_ids)

train_len = train.shape[0]
test_len = test.shape[0]
print(train_len)
print(train.shape)

train.describe()

<h5>count:每一列非空值的数量</h5>
<h5>mean: 每一列的平均值</h5>
<h5>std:每一列的标准差</h5>
<h5>min：最小值</h5>
<h5>25%：25%分位数，排序之后排在25%位置的数</h5>
<h5>50%：50%分位数</h5>
<h5>75%：75%分位数</h5>
<h5>max:最大值</h5>
<h5></h5>
<h4>我们由mean行看到多重疾病标签的图像比其他标签的图像要少得多。因此不能以原始数据形式加载图像，这里将使用scikitlearn随机地过采样，这样就可以修复这个类的不平衡。</h4>
<h5>现在来加载图片</h5>

In [ ]:
print("Shape of train data: " + str(train.shape))
print("Shape of test data: " + str(test.shape))

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from tqdm.notebook import tqdm

path = '../input/plant-pathology-2020-fgvc7/images/'
size = 224

train_images = np.ndarray(shape=(train_len, size, size, 3))
for i in tqdm(range(train_len)):
  img = load_img(path + f'Train_{i}.jpg', target_size=(size, size))
  train_images[i] = np.uint8(img_to_array(img))

test_images = np.ndarray(shape=(test_len, size, size, 3))
for i in tqdm(range(test_len)):
  img = load_img(path + f'Test_{i}.jpg', target_size=(size, size))
  test_images[i] = np.uint8(img_to_array(img))

train_images.shape, test_images.shape

<h4>现在让我们看一下分类好的图片</h4>
<h5></h5>
<h5>训练集的前四张图片：</h5>

In [ ]:
for i in range(4):
	plt.subplot(220 + 1 + i)
# 	plt.title(train['image_id'][i])
	plt.imshow(np.uint8(train_images[i]), interpolation = 'nearest', aspect='auto')
plt.show()
plt.savefig('train_images.png')

测试集的前四张图片：

In [ ]:
for i in range(4):
	plt.subplot(220 + 1 + i)
	plt.title(test['image_id'][i])
	plt.imshow(np.uint8(test_images[i]), interpolation = 'nearest', aspect='auto')
plt.show()
plt.savefig('test_images.png')

下面，将训练集分成训练和验两部分数据

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_images, target.to_numpy(), test_size=0.1, random_state=289) 

x_train.shape, x_test.shape, y_train.shape, y_test.shape

现在利用RandomOverSampler解决样本不平衡的问题

In [ ]:
from imblearn.over_sampling import RandomOverSampler #过采样处理

ros = RandomOverSampler()
x_train, y_train = ros.fit_resample(x_train.reshape((-1, size * size * 3)), y_train)
x_train = x_train.reshape((-1, size, size, 3))
x_train.shape, y_train.shape

<h5>现在为进入MobileNetV3模型准备数据。</h5>
<h5>在这里，我使用ImageDataGenerator通过使用参数来旋转、水平翻转和垂直翻转，也为我们提供了更多的图像。</h5>

In [ ]:
# samplewise_center：布尔值，使输入数据的每个样本均值为0
# samplewise_std_normalization：布尔值，将输入的每个样本除以其自身的标准差
from keras_preprocessing.image import ImageDataGenerator

batch_size = 8

train_datagen = ImageDataGenerator(samplewise_center = True,
                                   samplewise_std_normalization = True,
                                   horizontal_flip = True,
                                   vertical_flip = True,
                                   rotation_range=70)

train_generator = train_datagen.flow(
    x = x_train, 
    y = y_train,
    batch_size = batch_size)

validation_datagen = ImageDataGenerator(samplewise_center = True,
                                        samplewise_std_normalization = True)

validation_generator = validation_datagen.flow(
    x = x_test, 
    y = y_test,
    batch_size = batch_size)

让我们看看图像处理后的样子，以及它们进入模型后的样子。

In [ ]:
idx = np.random.randint(8)
print(idx)
x, y = train_generator.__getitem__(idx)
print(len(x))
print(len(y))
plt.title(y[idx])
plt.imshow(x[idx])
plt.show()

<h5>在这里建立模型。我将使用预先训练好的MobileNetV3Large进行深度CNN，然后将其输入到一个Dense层中预测4个类。</h5>
<h5>这里使用损失函数KL散度、Adam优化器和精度度量进行编译。</h5>

In [ ]:
def create_model():
    pre_trained = tf.keras.applications.MobileNetV3Large(input_shape=(size, size, 3), weights='imagenet', include_top=False)

    model = tf.keras.Sequential([
      pre_trained,
        tf.keras.layers.GlobalAveragePooling2D(),
#       tf.keras.layers.Flatten(),
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.Dense(4, activation='softmax')
      ])
    model.compile(
        loss = 'kullback_leibler_divergence', 
        optimizer = 'adam', 
        metrics = ['accuracy'])
    return model

model = create_model()

model.summary()

现在定义一些模型参数并设置一些回调,并开始训练模型

In [ ]:
epochs = 150
print(batch_size)
steps_per_epoch = x_train.shape[0] // batch_size
validation_steps = x_test.shape[0] // batch_size
print(steps_per_epoch)

In [ ]:
# EarlyStopping是用于提前停止训练的callbacks。当训练集上的loss不在减小（即减小的程度小于某个阈值）的时候停止继续训练。
# patience: 当early stop被激活(如发现loss相比上一个epoch训练没有下降)，则经过patience个epoch后停止训练
# restore_best_weights：是否从时期以受监视数量的最佳值恢复模型权重。如果为False，则使用在训练的最后一步获得的模型权重。
es = tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, verbose=1)
# 该回调函数将在每个epoch后保存模型到filepath
# verbose：信息展示模式，0或1。为1表示输出epoch模型保存信息，默认为0表示不输出该信息
mc = tf.keras.callbacks.ModelCheckpoint('model.hdf5', save_best_only=True, verbose=0)
# 在训练过程中如果出现了损失平台（loss plateau），即损失率不怎么变化时，改变学习率
rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, verbose=1)

start_lr = 0.00001
min_lr = 0.00001
max_lr = 0.00005
rampup_epochs = 40
sustain_epochs = 20
exp_decay = .8

def lrfn(epoch):
  if epoch < rampup_epochs:
    return (max_lr - start_lr)/rampup_epochs * epoch + start_lr
  elif epoch < rampup_epochs + sustain_epochs:
    return max_lr
  else:
    return min_lr
# 学习率调度函数LearningRateScheduler
lr = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=True)

rang = np.arange(epochs)
y = [lrfn(x) for x in rang]
plt.plot(rang, y)
print('Learning rate per epoch:')

In [ ]:
history = model.fit(
    x = train_generator,  
    validation_data = validation_generator,
    epochs = epochs,
    steps_per_epoch = steps_per_epoch,
    validation_steps = validation_steps,
    verbose=1,
    callbacks=[es, lr, mc, rlr])

下面我们来看一看预测准确率和损失度

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [ ]:
# samplewise_center：布尔值，使输入数据的每个样本均值为0
# samplewise_std_normalization：布尔值，将输入的每个样本除以其自身的标准差
test_datagen = ImageDataGenerator(samplewise_center = True,
                                 samplewise_std_normalization = True)

test_generator = test_datagen.flow(
    x = test_images,
    shuffle = False)

In [ ]:
probabilities = model.predict(test_generator, steps = len(test_generator))
print(probabilities)

In [ ]:
def plot_image(i):
    plt.title(test['image_id'][i])
    plt.imshow(np.uint8(test_images[i]),cmap=plt.cm.binary)

def plot_value_array(results):
    r= results
    c = ['healthy','multiple_diseases','rush','scab']
    plt.xticks(rotation=90)
    plt.bar(c,r)
    plt.ylim([0, 1])

让我们看一看前50张测试图片的预测结果

In [ ]:
for i in range(50):
    print(probabilities[i])
    plt.figure(figsize=(6, 3))
    plt.subplot(1, 2, 1)
    plot_image(i)
    plt.subplot(1, 2, 2)
    plot_value_array(probabilities[i])
    plt.show()